In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import mlflow
import mlflow.xgboost
import shap
import warnings

warnings.filterwarnings('ignore')

In [2]:
import boto3
import io

s3 = boto3.client(
    's3',
    endpoint_url='http://minio.minio.svc.cluster.local:9000',
    aws_access_key_id='minio',
    aws_secret_access_key='minio123',
)

In [3]:
obj_test = s3.get_object(Bucket='sample-data', Key='data/application_test.csv')
test_df = pd.read_csv(io.BytesIO(obj_test['Body'].read()))

In [4]:
obj_train = s3.get_object(Bucket='sample-data', Key='data/application_train.csv')
train_df = pd.read_csv(io.BytesIO(obj_train['Body'].read()))

In [5]:
# Get numerical and categorical columns
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# Remove 'SK_ID_CURR' and 'TARGET' from numerical features as they are identifiers/target
if 'SK_ID_CURR' in numerical_features:
    numerical_features.remove('SK_ID_CURR')
if 'TARGET' in numerical_features:
    numerical_features.remove('TARGET')

print("Categorical features:", len(categorical_features))
print(categorical_features)
print("\nNumerical features:", len(numerical_features))
print(numerical_features)

Categorical features: 16
['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

Numerical features: 104
['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'

In [6]:
from src.utils import PreprocessFeatureSelector

# Prepare the data (remove target column for features)
X_train = train_df.drop(columns=["TARGET"])
X_test = test_df.copy()
y_train = train_df["TARGET"].values

data_version = "v1"
save_train_data_path = "data/processed/train"
save_test_data_path = "data/processed/test"

# Create and run the PreprocessFeatureSelector wrapper.
pfs = PreprocessFeatureSelector(X_train, X_test, y_train,
                                categorical_features, numerical_features,
                                data_version=data_version,
                                save_train_data_path=save_train_data_path,
                                save_test_data_path=save_test_data_path)
selected_train, selected_test, selected_features, excluded_features = pfs.run()

print("Final Selected Features:", selected_features)
print("Total Excluded Features:", excluded_features)

(CVXPY) May 05 09:02:31 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Version of ortools (9.5.2237) is too old. Expected >= 9.7.0.')


2025-05-05 09:02:31.847 | INFO     | src.utils:run:119 - 🚀 Starting preprocessing and filtering...
2025-05-05 09:03:03.562 | INFO     | src.utils:run:123 - ✅ Preprocessing complete.
2025-05-05 09:03:03.566 | INFO     | src.utils:run:125 - 🧮 Features before filtering: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',

Final Selected Features: ['CODE_GENDER', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'ORGANIZATION_TYPE', 'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3']
Total Excluded Features: ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CNT_CHILDREN', 'AMT_ANNUITY', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMI

In [7]:
print("Target value counts:")
print(selected_train['TARGET'].value_counts())
print("\nPercentage distribution:")
print(selected_train['TARGET'].value_counts(normalize=True) * 100)

Target value counts:
TARGET
0    282686
1     24825
Name: count, dtype: int64

Percentage distribution:
TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64


In [8]:
from src.trainer import UnderWritingTrainer   

processed_train = "data/processed/train/processed_train_v1.csv"
processed_test  = "data/processed/test/processed_test_v1.csv"

trained_model = UnderWritingTrainer.train_model(
    model_name       = "lgbm",         
    processed_train  = processed_train,
    processed_test   = processed_test,
    version          = "v2",
    experiment_name  = "LGBM",     
    categorical_features = None
)


2025-05-05 09:06:26.009 | INFO     | src.trainer:train_model:191 - MLflow experiment = LGBM
2025-05-05 09:06:26.739 | INFO     | src.trainer:train_model:83 - Running Optuna search (10 trials)…
[I 2025-05-05 09:06:26,741] A new study created in memory with name: no-name-e1bed0ab-d7aa-4e64-a57d-9cac8b127e85


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 59
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2025-05-05 09:06:28,969] Trial 0 finished with value: 0.9192722306228964 and parameters: {'max_depth': 4, 'learning_rate': 0.0036479850578234104, 'n_estimators': 478, 'subsample': 0.7764231446406293, 'colsample_bytree': 0.99252028648306}. Best is trial 0 with value: 0.9192722306228964.


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 59
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482


[I 2025-05-05 09:06:31,576] Trial 1 finished with value: 0.9192722306228964 and parameters: {'max_depth': 5, 'learning_rate': 0.004990631038335131, 'n_estimators': 424, 'subsample': 0.9437106074112057, 'colsample_bytree': 0.5873458216615974}. Best is trial 0 with value: 0.9192722306228964.


...

[I 2025-05-05 09:06:42,171] Trial 8 finished with value: 0.9192722306228964 and parameters: {'max_depth': 8, 'learning_rate': 0.0035844768656730698, 'n_estimators': 267, 'subsample': 0.6276452464951567, 'colsample_bytree': 0.955600573559492}. Best is trial 0 with value: 0.9192722306228964.


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006078 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 59
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482


[I 2025-05-05 09:06:43,968] Trial 9 finished with value: 0.9192397118839731 and parameters: {'max_depth': 8, 'learning_rate': 0.021255678167405395, 'n_estimators': 276, 'subsample': 0.8666242214263773, 'colsample_bytree': 0.6862753290439687}. Best is trial 0 with value: 0.9192722306228964.
2025-05-05 09:06:43.971 | SUCCESS  | src.trainer:train_model:88 - Best params: {'max_depth': 4, 'learning_rate': 0.0036479850578234104, 'n_estimators': 478, 'subsample': 0.7764231446406293, 'colsample_bytree': 0.99252028648306}


[LightGBM] [Info] Number of positive: 24825, number of negative: 282686
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008428 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 59
[LightGBM] [Info] Number of data points in the train set: 307511, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

Successfully registered model 'v2_LightGBM'.
2025/05/05 09:09:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: v2_LightGBM, version 1
Created version '1' of model 'v2_LightGBM'.
2025-05-05 09:09:00.687 | SUCCESS  | src.trainer:log_model:175 - Model logged & registered as ‘v2_LightGBM’
